In [ ]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr, haven, googledrive)

In [ ]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [ ]:
  #Conectar a google drive
drive_auth(email = "claudiodanielpc@gmail.com",cache = FALSE)

In [ ]:
carpeta<-as_id(drive_get("https://drive.google.com/drive/folders/1TBfoik4w4DWhwFqWeZFgkZvrBdDbqzku"))


archivos<-drive_ls(carpeta)
print(archivos)

In [ ]:
archivos[1]

In [ ]:
estimacion_pobreza <- function(file_id, file_name) {
  # Descarga de datos
  local_path <- file.path("data", file_name)
  drive_download(as_id(file_id), path = local_path, overwrite = TRUE)

  # Leer datos
  datos <- read_dta(local_path)

  # Añadir año
  year <- str_extract(file_name, "\\d{4}")


  # Diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estima pobreza
  pobreza <- mydesign %>%
    #Filtrar datos para CDMX
  filter(entidad==9)%>%
    #Filtar grupo de población en pobreza
    #filter(pobreza_mmip == 1) %>%
    group_by(etnia,pobreza_mmip)%>%
    summarise(
      poblacion = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = as.numeric(year))

  return(pobreza)
}

In [ ]:
tabla_pobreza <- map_dfr(
  seq_len(nrow(archivos)),
  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x])
)%>%
  arrange(year)

print("Estimación de población en situación de pobreza en la CDMX")
tabla_pobreza